In [11]:
# needs to have a 7th column: loopID (easily done with awk '{print $0"\t"NR}')
inbedpe = '5k10k25k.bedpe.gz'
outbedpe = '5k10k25k_merged.bedpe'

In [12]:

bedpe = dict()

import gzip
with gzip.open(inbedpe, 'rb') as f:
    for line in f: 
        fields = line.rstrip().split("\t")   
        bedpe[fields[6]] = fields[0:6]
        bedpe[fields[6]].append('\n ')
    
'''
with open(inbedpe, 'rb') as f:
    for line in f: 
        fields = line.rstrip().split("\t")   
        bedpe[fields[6]] = fields[0:6]
        bedpe[fields[6]].append('\n ')
'''

'\nwith open(inbedpe, \'rb\') as f:\n    for line in f: \n        fields = line.rstrip().split("\t")   \n        bedpe[fields[6]] = fields[0:6]\n        bedpe[fields[6]].append(\'\n \')\n'

In [13]:
import subprocess

subprocess.call("zcat " + inbedpe + " | cut -f 1,2,3,7 | bedtools sort | bedtools merge -d 10 -c 4 -o distinct > /tmp/startMerged.bed", shell = True)
subprocess.call("zcat " + inbedpe + " | cut -f 4,5,6,7 |  bedtools sort | bedtools merge -d 10 -c 4 -o distinct > /tmp/stopMerged.bed", shell = True)

0

In [14]:
Alist = list()
Blist = list()

A = open('/tmp/startMerged.bed', 'r')
for line in A:
    fields = line.rstrip().split("\t")
    IDs = fields[3].split(',')
    IDs = map(int, IDs)
    IDs = set(IDs)
    Alist.append(IDs)
    
B = open('/tmp/stopMerged.bed', 'r')
for line in B:
    fields = line.rstrip().split("\t")
    IDs = fields[3].split(',')
    IDs = map(int, IDs)
    IDs = set(IDs)
    Blist.append(IDs)

In [15]:
Aset = map(set, Alist)
Bset = map(set, Blist)

In [16]:
MergableList = list()
for i in Aset:
    for j in Bset:
        O = i.intersection(j)
        if len(O) > 1:
            MergableList.append(O)

In [17]:
MergableList.sort(key=len, reverse=True)

In [18]:
MergableList

[{1940, 1942, 1960, 2014, 2020, 2028, 2034, 2093, 2151, 2183},
 {10675, 10739, 10812, 10865, 10946, 10967, 10979, 10983, 10989, 10990},
 {12759, 12835, 12871, 12876, 12883, 12899, 12932, 12950, 12963, 12986},
 {2719, 2733, 2760, 2778, 2782, 2790, 2794, 2796, 2803},
 {13363, 13389, 13412, 13468, 13497, 13498, 13533, 13566, 13694},
 {6254, 6394, 6400, 6423, 6439, 6464, 6474, 6488},
 {229, 267, 275, 349, 355, 421, 495, 510},
 {2140, 2255, 2280, 2287, 2302, 2319, 2322, 2335},
 {3241, 3315, 3373, 3406, 3423, 3736, 3738, 3760},
 {3387, 3467, 3475, 3579, 3596, 3599, 3618, 3640},
 {4213, 4268, 4323, 4340, 4382, 4404, 4489, 4521},
 {8731, 8741, 8820, 8835, 8859, 8869, 8893, 8898},
 {6781, 6793, 6796, 6812, 6820, 6832, 6852, 6859},
 {9482, 9488, 9654, 9661, 9769, 9810, 9906, 10035},
 {9264, 9356, 9357, 9373, 9398, 9618, 9678, 9721},
 {11828, 11903, 11948, 11963, 11977, 11996, 11997, 12003},
 {1022, 1054, 1142, 1178, 1247, 1387, 1457},
 {1952, 2091, 2148, 2163, 2240, 2258, 2270},
 {2993, 3014, 30

In [31]:
from numpy import mean
OUT = open(outbedpe, 'w')
OUT.write('\t'.join(['CHR1', 'X1', 'X2','CHR2','Y1','Y2','\n']))
for e, SET in enumerate(MergableList):
    if e > 10: 
        exit
    #lowestX1 = 0
    #higestY2 = 0
    #lowestX2 = 0
    #higestY1 = 0
    c = 'a'
    X1  = list()
    X2  = list()
    Y1  = list()
    Y2  = list()
    for E, ID in enumerate(SET):
        ID = str(ID)
        c = bedpe[ID][0]
        X1.append(int(bedpe[ID][1]))
        Y1.append(int(bedpe[ID][4]))
        X2.append(int(bedpe[ID][2]))
        Y2.append(int(bedpe[ID][5]))
    OUT.write('\t'.join(     map(str,[c, int(mean(X1)), int(mean(X2)),c,int(mean(Y1)),int(mean(Y2)),'\n'])))    
    # 
    #    if E == 0:
    #        c = bedpe[ID][0]
    #        lowestX1 = int(bedpe[ID][1])
    #        lowestY1 = int(bedpe[ID][4])
    #        higestX2 = int(bedpe[ID][2] )
    #        higestY2 = int(bedpe[ID][5] )
    #    else:
    #        if int(bedpe[ID][1]) < lowestX1:
    #            lowestX1 = int(bedpe[ID][1])
    #        if int(bedpe[ID][5]) > higestY2:
    #            higestY2 = int(bedpe[ID][5] )
    #        if int(bedpe[ID][1]) < lowestX2:
    #            lowestX2 = int(bedpe[ID][2])
    #        if int(bedpe[ID][4]) > higestY1:
    #            higestY1 = int(bedpe[ID][4] )
    #OUT.write('\t'.join(     map(str,[c, lowestX1, higestX2,c,lowestY1,higestY2,'\n'])))

In [32]:
for ID in set(bedpe.keys()).symmetric_difference(set(map(str,[item for sublist in MergableList for item in sublist]))):
    OUT.write('\t'.join(bedpe[ID]))

In [33]:
OUT.close()